## University of California - Los Angeles Crawler

Imports.

In [6]:
import pandas as pd
import numpy as np
import re
import urllib.request #handles urls
import urllib.parse 
import linkGrabber #extracts urls
import json #encodes/decodes json 
import csv 
import requests #downloads a webpage to scrape
from bs4 import BeautifulSoup, NavigableString, Tag #beautifulsoup pulls data from HTML
import nltk #NLP tasks
from nltk import word_tokenize
from nltk.stem import PorterStemmer #removes word endings
stemmer = PorterStemmer()

Keyword preprocessing and url list of relevant catalog years; 2019-20 only since 18-19 cannot be found. Also the backend for the search url.

In [7]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
    return (keyword)

#course catalog URLs - 2 academic years 
urls = ['https://www.registrar.ucla.edu/Academics/Course-Descriptions/Course-Details?qry=']

#list of all the departments to search through 
search = '&funsel=1'

Creation of normative and technical keywords lists, the same as in example crawler.

In [8]:
#import keywords
keywords = pd.read_csv("../keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


Extraction process for University of California Los Angeles:
1. Loop through each years' catalog.
2. Append keyword to url, then append the rest of the search string to make search page url, note that the search function for UCLA searches descriptions as well as titles.
3. Make a list of all the courses by selecting the class id media-body and loop through them.
4. Check to make sure the keyword is in the course title.
5. If the keyword is in the title, then assign every element of the data columns that can be located.

Data columns are defined in the same way as below and have the same anatomy for each course:
* The course title - after the first instance of '.' in the title, which is the second element in the course list: `title`
* The department and course number - before the first occurance of '.' in the title: `dept_num`
* The course description - the fourth list element for the course: `description`
* The number of credits for the course - the third element in the course list: `credits`
* The course instructor - school does not list in catalog: `instructor`
* The link to the course syllabus (if applicable) - school does not list in catalog: `syllabus`
* The university the course is extracted from - all from the same university: `university`
* The term that the course is offered during (fall, spring, summer / year) - there is only one year found: `term`
* The keyword that triggered the extraction (this is for auditing purposes): `keyword`

In [9]:
#init dfs
angeles = pd.DataFrame(columns=['title','dept_num','description','credits','instructor',
                                'syllabus','university','term','keyword'])
titles = []
dept_nums = []
descs = []
credit = []
profs = []
syllabi = []
uni = []
term = []     
keyword = []
URL = []

The extraction process. The process to create the table is kept the same as the example crawler, just as a loop on it's own after all the titles, credits, etc. are all gathered.

In [10]:
#looping through each years catalog
for url in urls:
    for word in normative:
        page_link = url + word + search
        page_response = requests.get(page_link)
        soup = BeautifulSoup(page_response.content, 'html.parser')
        courses = [p.get_text().split('\n') for p in soup.select(".media-body")]
        for crs in courses:
            title = crs[1]
            if word in title.lower():
                URL.append(page_link)
                titles.append(title[title.find('.')+2:])
                dept_nums.append(title[:title.find('.')])
                if(crs[3]==''): descs.append('No description available.')
                else: descs.append(crs[3])
                credit.append(crs[2])
                profs.append('Not Listed')
                syllabi.append('Not Listed')
                uni.append('University of California - Los Angeles')
                term.append('2019-20')
                keyword.append(word)
            
for a,b,c,d,e,f,g,h,i,j in zip(titles,dept_nums,descs,credit,profs,syllabi,uni,term,keyword,URL):
    angeles = angeles.append({'title': a, 
                              'dept_num': b,
                              'description': c,
                              'credits': d,
                              'instructor': e,
                              'syllabus': f,
                              'university': g,
                              'term': h,
                              'keyword': i,
                              'URL': j}, ignore_index=True)

angeles

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword,URL
0,Healthcare Financial Accounting,403,"Lecture, four hours. Introduction to basic con...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
1,Accounting for Lawyers,234,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
2,Accounting and Financial Skills for Lawyers,434,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
3,"Accounting, Finance, and the law",604,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
4,Civil Rights and Policy Accountability Clinic,708,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
5,Principles of Accounting,1A,"Lecture, three hours; discussion, one hour. No...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
6,Principles of Accounting,1B,"Lecture, three hours; discussion, one hour. Re...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
7,Intermediate Financial Accounting I,120A,"Lecture, four hours. Requisite: course 1B. Int...",Units: 5,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
8,Intermediate Financial Accounting II,120B,"Lecture, four hours. Requisite: course 120A. I...",Units: 5,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...
9,Ethical Leadership in Accounting,121,"Lecture, seven and one half hours. Not open to...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...


Post filtering of course. Code is identical to that of example crawler.

In [ ]:
exceptions = angeles.loc[(angeles['keyword']=='privac') | (angeles['keyword'] =='secur')]
exceptions

In [32]:
#loop through technical keyword list, extract relevant titles
new = []
for row in angeles.title:
    flag = False
    for word in technical:
        if word in row.lower():
            flag = True
            new.append(word)
            break
    if flag == False: 
        new.append('')
print(len(new))
print(len(angeles))
print(new)

3511
3511
['', '', '', '', '', '', '', '', '', '', '', '', '', 'inform', 'program', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'analyt', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'technolog', '', '', '', '', 'inform', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [33]:
angeles['new'] = new
angeles

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword,URL,new
0,Healthcare Financial Accounting,403,"Lecture, four hours. Introduction to basic con...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
1,Accounting for Lawyers,234,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
2,Accounting and Financial Skills for Lawyers,434,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
3,"Accounting, Finance, and the law",604,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
4,Civil Rights and Policy Accountability Clinic,708,No description available.,Units: 1,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
5,Principles of Accounting,1A,"Lecture, three hours; discussion, one hour. No...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
6,Principles of Accounting,1B,"Lecture, three hours; discussion, one hour. Re...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
7,Intermediate Financial Accounting I,120A,"Lecture, four hours. Requisite: course 1B. Int...",Units: 5,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
8,Intermediate Financial Accounting II,120B,"Lecture, four hours. Requisite: course 120A. I...",Units: 5,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,
9,Ethical Leadership in Accounting,121,"Lecture, seven and one half hours. Not open to...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,account,https://www.registrar.ucla.edu/Academics/Cours...,


In [8]:
angelesNew = angeles[(angeles['new']!='') | (angeles['keyword']=='privac') | (angeles['keyword'] =='secur')]
cornellNew

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword,URL
13,Information Technology in Accounting,142A,"(Formerly numbered 142.) Lecture, seven and on...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"account,technolog",https://www.registrar.ucla.edu/Academics/Cours...
14,"Communication Technology, Programming, and Acc...",142B,"Lecture, six hours. Preparation: intermediate ...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"account,technolog",https://www.registrar.ucla.edu/Academics/Cours...
216,Ethics and Impact of Technology on Society,181EW,"Lecture, five hours; discussion, three hours; ...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"ethic,technolog",https://www.registrar.ucla.edu/Academics/Cours...
552,"Ancient and Historic Metals: Corrosion, Techno...",C180,"Seminar, four hours; laboratory, four hours. O...",Units: 6,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...
554,"Ancient and Historic Metals: Corrosion, Techno...",C280,"Seminar, four hours; laboratory, four hours. O...",Units: 6,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...
703,"Youth Culture and MTV: Technology, History, Po...",98TA,"Seminar, three hours. Requisite: satisfaction ...",Units: 5,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...
793,Introduction to Historical Practice: Variable ...,97I,"Seminar, three hours. Discussion classes of no...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...
957,Variable Topics Historiography Proseminar: Sci...,187I,"Seminar, three hours. Proseminar on historiogr...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...
972,Capstone Seminar: History--Science/Technology,191I,"Seminar, three hours. Designed for seniors. Li...",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...
1001,Advanced Historiography: Science/Technology,200O,"Seminar, three hours. May be repeated for credit.",Units: 4,Not Listed,Not Listed,University of California - Los Angeles,2019-20,"histor,technolog",https://www.registrar.ucla.edu/Academics/Cours...


Exporting of code to csv.

In [9]:
#export as csv
angeles.to_csv('27 - University of California Los Angeles.csv')